In [1]:
import json
import os
import random
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import scipy.sparse as sp

random.seed(42)
np.random.seed(42)

In [6]:
OUTPUT_DATA_DIR = "./output_data/"

val_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_validation.csv")
test_df = pd.read_csv(OUTPUT_DATA_DIR+"interactions_testing.csv")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
def load_mapping(mapping_file):
    """Loads the mapping from `mapping_file`.
    
    Parameters
    ----------
    mapping_file: str
        The name of the mapping file to import.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame created from the mapping.
    
    """
    return pd.read_csv(os.path.join("mappings", "{}.csv".format(mapping_file)))

In [5]:
user_map = load_mapping("user_map")
book_map = load_mapping("book_map")

In [7]:
val_df['book_id'] = val_df['book_id'].apply(lambda x: str(x))
test_df['book_id'] = test_df['book_id'].apply(lambda x: str(x))
book_map['book_id'] = book_map['book_id'].apply(lambda x: str(x))

In [8]:
val_df = pd.merge(val_df, user_map, how="left", on=["user_id"])
val_df = pd.merge(val_df, book_map, how="left", on=["book_id"])

test_df = pd.merge(test_df, user_map, how="left", on=["user_id"])
test_df = pd.merge(test_df, book_map, how="left", on=["book_id"])

In [10]:
pd.set_option('display.max_columns', None)

In [12]:
val_df['max_stage_vali'] = val_df['read'] + val_df['rated'] + val_df['recommended']
test_df['max_stage_test'] = test_df['read'] + test_df['rated'] + test_df['recommended']

In [13]:
val_df = val_df[['user_number', 'book_number', 'max_stage_vali']]
val_df.columns = ['user_id', 'item_id_vali', 'max_stage_vali']

test_df = test_df[['user_number', 'book_number', 'max_stage_test']]
test_df.columns = ['user_id', 'item_id_test', 'max_stage_test']

In [16]:
val_test_df = pd.merge(val_df, test_df, how='inner', on=['user_id'])
val_test_df = val_test_df[['user_id', 'item_id_vali', 'max_stage_vali', 'item_id_test', 'max_stage_test']]

In [18]:
val_test_df.to_csv(OUTPUT_DATA_DIR+"goodreads.test.csv", index=False)

In [19]:
val_test_df

,user_id,item_id_vali,max_stage_vali,item_id_test,max_stage_test
0,96915,7,3,60,3
1,30739,12,3,873,3
2,20160,26,0,2,3
3,15499,30,3,1212,0
4,79298,42,3,1163,0
...,...,...,...,...,...
34625,64651,1158,3,539,3
34626,81665,61,3,1691,3
34627,96990,73,3,106,3
34628,65900,10,3,305,2
